In [7]:
import os
import pandas as pd
import zipfile

In [8]:
EXT_DIR = "/media/reggie/reg_ext/EDGAR/FSANDS"

In [9]:
def unzip_fsands(year, quarter):
    target_filename = os.path.join(EXT_DIR, 'zips',str(year)+'q'+str(quarter)+'.zip')
    target_dir = os.path.join(EXT_DIR, 'unzipped',str(year)+'q'+str(quarter))
    print(target_dir)
    if not os.path.isdir(target_dir):
        os.makedirs(target_dir)
    with zipfile.ZipFile(target_filename, 'r') as zip_ref:
        zip_ref.extractall(target_dir)

In [10]:
def extract_fsands(filepath):
    print(filepath)

    with open(filepath, "r", encoding='latin1') as infile:
        contents = infile.readlines()

    data = [line.strip().split("\t") for line in contents]
    bads = []
    goods = []
    for line in data:
        if len(line) == 20:
            goods.append(line)
        else:
            bads.append(line)

    with open(filepath.replace(".tsv","") + "-clean.tsv", "w") as outfile:
        outfile.writelines(["\t".join(line)+"\n" for line in goods])
    with open(filepath.replace(".tsv","") + "-bad.tsv", "w") as outfile:
        outfile.writelines(["\t".join(line)+"\n" for line in bads])
        
    print(len(goods), " goods and ",len(bads),"bads in ",filepath)

In [16]:
def get_sap(txt_submissions, form_type, year, quarter):
    x = pd.read_csv(txt_submissions, sep="\t", low_memory=False)
    ## Merge CIK numbers
    subs = pd.read_csv(EXT_DIR+"/unzipped/{}q{}/sub.tsv".format(year, quarter), sep="\t")
    subs_10k = subs[subs["form"].apply(lambda x: str(x).find(form_type) > -1)]
    ciks = subs_10k[["adsh","cik","sic","form","period","fye","fp","filed"]]
    sap = x[x["tag"] == "SignificantAccountingPoliciesTextBlock"]
    sap = sap.merge(ciks)
    sap.to_csv(EXT_DIR + "/SAP/{}q{}.tsv".format(year, quarter), sep="\t", index=False)

In [17]:
for year in [2010]:
    for quarter in range(1,5):
        unzip_fsands(year, quarter)
        txt_submissions = os.path.join(EXT_DIR, "unzipped/{}q{}/txt.tsv".format(year,quarter))
        try:
            extract_fsands(txt_submissions)
            get_sap(txt_submissions.replace(".tsv","-clean.tsv"), "10-K",year, quarter)
        except:
            continue

/media/reggie/reg_ext/EDGAR/FSANDS/unzipped/2010q1
/media/reggie/reg_ext/EDGAR/FSANDS/unzipped/2010q1/txt.tsv
15580  goods and  685 bads in  /media/reggie/reg_ext/EDGAR/FSANDS/unzipped/2010q1/txt.tsv
/media/reggie/reg_ext/EDGAR/FSANDS/unzipped/2010q2
/media/reggie/reg_ext/EDGAR/FSANDS/unzipped/2010q2/txt.tsv
13869  goods and  687 bads in  /media/reggie/reg_ext/EDGAR/FSANDS/unzipped/2010q2/txt.tsv
/media/reggie/reg_ext/EDGAR/FSANDS/unzipped/2010q3
/media/reggie/reg_ext/EDGAR/FSANDS/unzipped/2010q3/txt.tsv
49341  goods and  1880 bads in  /media/reggie/reg_ext/EDGAR/FSANDS/unzipped/2010q3/txt.tsv
/media/reggie/reg_ext/EDGAR/FSANDS/unzipped/2010q4
/media/reggie/reg_ext/EDGAR/FSANDS/unzipped/2010q4/txt.tsv
54195  goods and  1993 bads in  /media/reggie/reg_ext/EDGAR/FSANDS/unzipped/2010q4/txt.tsv


In [19]:
sap = pd.read_csv(EXT_DIR+"/SAP/2010q1.tsv", sep="\t", low_memory=False)
print(sap.shape)
sap.head()

(277, 27)


,adsh,tag,version,ddate,qtrs,iprx,lang,dcml,durp,datp,...,footlen,context,value,cik,sic,form,period,fye,fp,filed
0,0000029915-10-000024,SignificantAccountingPoliciesTextBlock,us-gaap/2009,20091231,4,0,en-US,32767,0.010959,0.0,...,0,c00004,NOTE A  SUMMARY OF SIGNIFICANT ACCOUNTING POL...,29915,2821,10-K,20091231,1231,FY,20100219
1,0000040545-10-000010,SignificantAccountingPoliciesTextBlock,us-gaap/2009,20091231,4,0,en-US,32767,0.010959,0.0,...,0,FROM_Jan01_2009_TO_Dec31_2009,Notes to Consolidated Financial Statements Not...,40545,3600,10-K,20091231,1231,FY,20100219
2,0000060667-10-000064,SignificantAccountingPoliciesTextBlock,us-gaap/2009,20100131,4,0,en-US,32767,0.021918,2.0,...,0,FROM_Jan31_2009_TO_Jan29_2010,NOTE 1: Summary of Significant Accounting Poli...,60667,5211,10-K/A,20100131,131,FY,20100330
3,0000065984-10-000041,SignificantAccountingPoliciesTextBlock,us-gaap/2009,20091231,4,0,en-US,32767,0.010959,0.0,...,0,c00047,NOTE 1. SUMMARY OF SIGNIFICANT ACCOUNTING POLI...,65984,4911,10-K/A,20091231,1231,FY,20100226
4,0000067716-10-000030,SignificantAccountingPoliciesTextBlock,us-gaap/2009,20091231,4,0,en-US,32767,0.010959,0.0,...,0,c00007,Note 1  Summary of Significant Accounting Pol...,67716,1400,10-K,20091231,1231,FY,20100217
